FIRST TRY RANDOM FOREST BASED ON rndm-frst-model.ipynb


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt #to visualise
import seaborn as sns

import nltk #to vectorizer
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer #to find the similarity

#Random Forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier #not regression
from sklearn.metrics import classification_report, accuracy_score

import re

In [2]:
laptop = pd.read_csv('dataset/laptops.csv')

In [3]:
laptop.shape

(796, 16)

# DATA PREPROCESS
-remove null and duplicate


In [4]:
print('CHECK NULL VALUE: ', laptop.isnull().sum())
print('CHECK MISSING VALUE: ', laptop.duplicated().sum())

CHECK NULL VALUE:  laptop_id            0
name                 0
price                0
processor            0
ram                  0
os                   0
storage              0
img_link             0
display              0
rating             266
no_of_ratings      266
no_of_reviews      266
laptop_brand         0
os_brand             0
processor_brand      0
usecases             0
dtype: int64
CHECK MISSING VALUE:  0


In [5]:
laptop.dropna(inplace=True)

In [6]:
print('DF SHAPE: ', laptop.shape)
print('CHECK NULL VALUE: ', laptop.isnull().sum())
print('CHECK MISSING VALUE: ', laptop.duplicated().sum())

DF SHAPE:  (530, 16)
CHECK NULL VALUE:  laptop_id          0
name               0
price              0
processor          0
ram                0
os                 0
storage            0
img_link           0
display            0
rating             0
no_of_ratings      0
no_of_reviews      0
laptop_brand       0
os_brand           0
processor_brand    0
usecases           0
dtype: int64
CHECK MISSING VALUE:  0


---END OF DATA PREPROCESS---

# DATA TRANSFORM

In [7]:
#laptop['display'] = laptop['display'].astype(str)
print(laptop.dtypes)

laptop_id            int64
name                object
price                int64
processor           object
ram                 object
os                  object
storage             object
img_link            object
display            float64
rating             float64
no_of_ratings      float64
no_of_reviews      float64
laptop_brand        object
os_brand            object
processor_brand     object
usecases            object
dtype: object


In [8]:
#laptop['display'] = pd.to_numeric(laptop['display'], errors='coerce')

laptop['storage'] = pd.to_numeric(laptop['storage'], errors='coerce')
print(laptop.dtypes)

laptop_id            int64
name                object
price                int64
processor           object
ram                 object
os                  object
storage            float64
img_link            object
display            float64
rating             float64
no_of_ratings      float64
no_of_reviews      float64
laptop_brand        object
os_brand            object
processor_brand     object
usecases            object
dtype: object


# A. CURRENCY

-change from Rupee to MYR

-drop rupee column and change with rm


In [9]:
laptop['price'] = laptop['price'] * 0.055 

In [10]:
laptop.shape

(530, 16)

In [11]:
#Drop 4 columns
laptop.drop(columns=['os', 'no_of_ratings', 'no_of_reviews', 'rating'], inplace=True) #To drop column
laptop.rename(columns={'os_brand': 'os'}, inplace=True)

In [12]:
laptop.shape

(530, 12)

In [13]:
laptop.columns

Index(['laptop_id', 'name', 'price', 'processor', 'ram', 'storage', 'img_link',
       'display', 'laptop_brand', 'os', 'processor_brand', 'usecases'],
      dtype='object')

In [14]:
#laptop['usecases'].unique() #to display all usecase without total them
laptop['usecases'].value_counts()

usecases
 Gaming                      152
 Business/Professional       108
 Home/Everyday use            81
 Budget Friendly              55
 Student/Education            47
 Creative/Design              37
 Multimedia/Entertainment     31
Ultra-portable                19
Name: count, dtype: int64

# B. RAM

In [15]:
laptop['ram'].value_counts().head()

ram
8 GB DDR4 RAM       264
16 GB DDR4 RAM      123
4 GB DDR4 RAM        34
16 GB DDR5 RAM       23
8 GB LPDDR4X RAM     18
Name: count, dtype: int64

In [16]:
#need to extract value of each RAM and create column for each value/category
#we extracts numeric values from the text, and convert into integer
laptop['ram'] = laptop['ram'].str.extract('(\d+)').astype(int)

In [17]:
laptop['ram'].value_counts().head()

ram
8     299
16    173
4      51
32      7
Name: count, dtype: int64

# C. STORAGE

In [18]:
#laptop['storage'] = laptop['storage'].str.extract('(\d+)').astype(int)

In [19]:
laptop['ram'].head()

0    16
1    16
2     8
3     4
4     8
Name: ram, dtype: int32

In [20]:
laptop['storage'] = laptop['storage'].apply(lambda x: 1000 if x == 1 else x)
laptop['storage'] = laptop['storage'].apply(lambda x: 2000 if x == 2 else x)

In [21]:
laptop['storage'].value_counts().head()

Series([], Name: count, dtype: int64)

# D. DISPLAY

In [22]:
sorted(laptop['display'].unique())

[11.6,
 13.3,
 13.4,
 13.6,
 14.0,
 14.1,
 14.2,
 14.96,
 15.0,
 15.6,
 16.0,
 16.1,
 16.2,
 16.6,
 17.3,
 35.0]

# E. OS

In [23]:
laptop['os'].value_counts().head()

os
Windows     486
MacOS        25
ChromeOS     14
DOS           5
Name: count, dtype: int64

In [24]:
laptop['os_windows'] = laptop['os'].apply(lambda x: 1 if 'windows' in str(x).lower() else 0)
laptop['os_mac'] = laptop['os'].apply(lambda x: 1 if 'macos' in str(x).lower() else 0)

laptop[['os', 'os_windows', 'os_mac']]

,os,os_windows,os_mac
0,Windows,1,0
1,Windows,1,0
2,Windows,1,0
3,DOS,0,0
4,MacOS,0,1
...,...,...,...
786,Windows,1,0
788,Windows,1,0
791,Windows,1,0
793,Windows,1,0


# F. PROCESSOR / CPU


In [25]:
laptop['processor'].value_counts().head()

processor
Intel Core i3 Processor (11th Gen)    62
Intel Core i5 Processor (11th Gen)    57
AMD Ryzen 5 Hexa Core Processor       57
Intel Core i5 Processor (12th Gen)    48
AMD Ryzen 7 Octa Core Processor       36
Name: count, dtype: int64

In [26]:
laptop['processor_brand'].value_counts().head()

processor_brand
Intel       355
AMD         149
Apple M1     23
Qualcomm      1
Apple M2      1
Name: count, dtype: int64

In [27]:
laptop.head()

,laptop_id,name,price,processor,ram,storage,img_link,display,laptop_brand,os,processor_brand,usecases,os_windows,os_mac
0,0,Lenovo Intel Core i5 11th Gen,3464.45,Intel Core i5 Processor (11th Gen),16,NaN,https://rukminim1.flixcart.com/image/312/312/x...,15.6,Lenovo,Windows,Intel,Business/Professional,1,0
1,1,ASUS ROG Zephyrus G15 (2022) with 90Whr Batter...,8909.45,AMD Ryzen 9 Octa Core Processor,16,NaN,https://rukminim1.flixcart.com/image/312/312/l...,15.6,ASUS,Windows,AMD,Gaming,1,0
2,2,ASUS VivoBook 15 (2022) Core i3 10th Gen,1869.45,Intel Core i3 Processor (10th Gen),8,NaN,https://rukminim1.flixcart.com/image/312/312/x...,15.6,ASUS,Windows,Intel,Home/Everyday use,1,0
3,3,Lenovo Athlon Dual Core,1044.45,AMD Athlon Dual Core Processor,4,NaN,https://rukminim1.flixcart.com/image/312/312/x...,14.0,Lenovo,DOS,AMD,Budget Friendly,0,0
4,4,APPLE 2020 Macbook Air M1,4784.45,Apple M1 Processor,8,NaN,https://rukminim1.flixcart.com/image/312/312/k...,13.3,APPLE,MacOS,Apple M1,Multimedia/Entertainment,0,1


In [28]:
#need to extract and create new columns for unique processor
#display all distinct CPU
#laptop['processor'].unique()
#display only this CPU
#laptop[laptop['processor'].str.contains('Apple', case=False, na=False)]['processor'].unique()
#display other than selective CPU
#laptop[~laptop['processor'].str.contains('Apple|Intel Core', case=False, na=False)]['processor'].unique()

In [29]:
#CPU Intel
laptop['i3'] = laptop['processor'].apply(lambda x: 1 if 'i3' in str(x).lower() else 0)
laptop['i5'] = laptop['processor'].apply(lambda x: 1 if 'i5' in str(x).lower() else 0)
'''laptop['i7'] = laptop['processor'].apply(lambda x: 1 if 'i7' in str(x).lower() else 0)
laptop['i9'] = laptop['processor'].apply(lambda x: 1 if 'i9' in str(x).lower() else 0)'''
#CPU Ryzen
laptop['r3'] = laptop['processor'].apply(lambda x: 1 if 'Ryzen 3' in str(x).lower() else 0)
laptop['r5'] = laptop['processor'].apply(lambda x: 1 if 'Ryzen 5' in str(x).lower() else 0)
'''laptop['r7'] = laptop['processor'].apply(lambda x: 1 if 'Ryzen 7' in str(x).lower() else 0)
laptop['r9'] = laptop['processor'].apply(lambda x: 1 if 'Ryzen 9' in str(x).lower() else 0)
#CPU MAC
laptop['m1'] = laptop['processor'].apply(lambda x: 1 if 'M1' in str(x).lower() else 0)
laptop['m2'] = laptop['processor'].apply(lambda x: 1 if 'M2' in str(x).lower() else 0)
#CPU Others
laptop['sd'] = laptop['processor'].apply(lambda x: 1 if 'Snapdragon' in str(x).lower() else 0)
laptop['mdtk'] = laptop['processor'].apply(lambda x: 1 if 'MediaTek' in str(x).lower() else 0)
laptop['2c'] = laptop['processor'].apply(lambda x: 1 if 'Dual Core' in str(x).lower() else 0)
laptop['4c'] = laptop['processor'].apply(lambda x: 1 if 'Quad Core' in str(x).lower() else 0)
laptop['6c'] = laptop['processor'].apply(lambda x: 1 if 'Hexa Core' in str(x).lower() else 0)
laptop['8c'] = laptop['processor'].apply(lambda x: 1 if 'Octa Core' in str(x).lower() else 0)'''

#display selective columns
#laptop[['name', 'usecases']].head() 
laptop.head()

,laptop_id,name,price,processor,ram,storage,img_link,display,laptop_brand,os,processor_brand,usecases,os_windows,os_mac,i3,i5,r3,r5
0,0,Lenovo Intel Core i5 11th Gen,3464.45,Intel Core i5 Processor (11th Gen),16,NaN,https://rukminim1.flixcart.com/image/312/312/x...,15.6,Lenovo,Windows,Intel,Business/Professional,1,0,0,1,0,0
1,1,ASUS ROG Zephyrus G15 (2022) with 90Whr Batter...,8909.45,AMD Ryzen 9 Octa Core Processor,16,NaN,https://rukminim1.flixcart.com/image/312/312/l...,15.6,ASUS,Windows,AMD,Gaming,1,0,0,0,0,0
2,2,ASUS VivoBook 15 (2022) Core i3 10th Gen,1869.45,Intel Core i3 Processor (10th Gen),8,NaN,https://rukminim1.flixcart.com/image/312/312/x...,15.6,ASUS,Windows,Intel,Home/Everyday use,1,0,1,0,0,0
3,3,Lenovo Athlon Dual Core,1044.45,AMD Athlon Dual Core Processor,4,NaN,https://rukminim1.flixcart.com/image/312/312/x...,14.0,Lenovo,DOS,AMD,Budget Friendly,0,0,0,0,0,0
4,4,APPLE 2020 Macbook Air M1,4784.45,Apple M1 Processor,8,NaN,https://rukminim1.flixcart.com/image/312/312/k...,13.3,APPLE,MacOS,Apple M1,Multimedia/Entertainment,0,1,0,0,0,0


ASSIGN NEW USECASE

In [30]:
laptop.columns

Index(['laptop_id', 'name', 'price', 'processor', 'ram', 'storage', 'img_link',
       'display', 'laptop_brand', 'os', 'processor_brand', 'usecases',
       'os_windows', 'os_mac', 'i3', 'i5', 'r3', 'r5'],
      dtype='object')

In [31]:
'''def assign_usecase(row):
    # For GAMING: Display (>=15.6), CPU (i5, r5), RAM (>=16), Storage (>=512)
    if (row['display'] >= 15.6 and (row['i5'] == 1 or row['r5'] == 1) and row['ram'] >= 16 and row['storage'] >= 512):
        return "gaming"
    # For OFFICE: Display (<=15.6), CPU (i3, r3), RAM (8-16), Storage (256-1000)
    elif (row['display'] <= 15.6 and (row['i3'] == 1 or row['r3'] == 1) and 8 <= row['ram'] <= 16 and 256 <= row['storage'] <= 1000):
        return "office"
    # For HYBRID: Display (15.6 - 17.3), CPU (i5, r5), RAM (<=8), Storage (<=512)
    elif (15.6 <= row['display'] <= 17.3 and (row['i5'] == 1 or row['r5'] == 1) and row['ram'] <= 8 and row['storage'] <= 512):
        return "hybrid"
    # Default case (if no criteria match)
    return "unknown"'''

# Apply the function to the DataFrame
laptop['usecases'] = laptop.apply(assign_usecase, axis=1)


NameError: name 'assign_usecase' is not defined

In [284]:
import pickle

#laptop.to_csv('model/rf-dataset.csv', sep=',', index=False, encoding='utf-8')

In [285]:
laptop[['ram', 'storage', 'i3', 'i5', 'r3', 'r5']].head(10)

,ram,storage,i3,i5,r3,r5
0,16,NaN,0,1,0,0
1,16,NaN,0,0,0,0
2,8,NaN,1,0,0,0
3,4,NaN,0,0,0,0
4,8,NaN,0,0,0,0
5,4,NaN,0,0,0,0
6,8,NaN,1,0,0,0
7,8,NaN,0,1,0,0
8,8,NaN,0,0,0,0
9,16,NaN,0,1,0,0


In [286]:
laptop['usecases'].value_counts().head()

usecases
unknown    530
Name: count, dtype: int64

# MODEL CREATION

In [287]:
# Ensure laptop DataFrame is not empty and contains required columns
if 'ram' in laptop.columns and not laptop.empty:
    X = laptop[['ram', 'i3', 'i5', 'r3', 'r5']]
    y = laptop['usecases']
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
else:
    print("Error: DataFrame is empty or required columns are missing.")


In [288]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((424, 5), (106, 5), (424,), (106,))

In [289]:
# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Classification report for more detailed evaluation
print(classification_report(y_test, y_pred))

Accuracy: 1.0000
              precision    recall  f1-score   support

     unknown       1.00      1.00      1.00       106

    accuracy                           1.00       106
   macro avg       1.00      1.00      1.00       106
weighted avg       1.00      1.00      1.00       106



In [290]:
# Example of predicting the usecase for a new laptop based on user input
def predict_usecase(ram, processor, os):
    input_data = {
        'ram': [ram],
        'i3': [1 if 'i3' in processor.lower() else 0],
        'i5': [1 if 'i5' in processor.lower() else 0],
        'r3': [1 if 'ryzen 3' in processor.lower() else 0],
        'r5': [1 if 'ryzen 5' in processor.lower() else 0],
        'os': [os]
    }

    input_df = pd.DataFrame(input_data)

    # Predict the usecase for the given laptop specifications
    predicted_usecase = rf_model.predict(input_df)
    
    return predicted_usecase[0]

'''     'i7': [1 if 'i7' in processor.lower() else 0],
        'i9': [1 if 'i9' in processor.lower() else 0],
        'r7': [1 if 'ryzen 7' in processor.lower() else 0],
        'r9': [1 if 'ryzen 9' in processor.lower() else 0],
        'm1': [1 if 'm1' in processor.lower() else 0],
        'm2': [1 if 'm2' in processor.lower() else 0],
        'sd': [1 if 'snapdragon' in processor.lower() else 0],
        'mdtk': [1 if 'mediatek' in processor.lower() else 0],
        '2c': [1 if 'dual core' in processor.lower() else 0],
        '4c': [1 if 'quad core' in processor.lower() else 0],
        '6c': [1 if 'hexa core' in processor.lower() else 0],
        '8c': [1 if 'octa core' in processor.lower() else 0]
'''


"     'i7': [1 if 'i7' in processor.lower() else 0],\n        'i9': [1 if 'i9' in processor.lower() else 0],\n        'r7': [1 if 'ryzen 7' in processor.lower() else 0],\n        'r9': [1 if 'ryzen 9' in processor.lower() else 0],\n        'm1': [1 if 'm1' in processor.lower() else 0],\n        'm2': [1 if 'm2' in processor.lower() else 0],\n        'sd': [1 if 'snapdragon' in processor.lower() else 0],\n        'mdtk': [1 if 'mediatek' in processor.lower() else 0],\n        '2c': [1 if 'dual core' in processor.lower() else 0],\n        '4c': [1 if 'quad core' in processor.lower() else 0],\n        '6c': [1 if 'hexa core' in processor.lower() else 0],\n        '8c': [1 if 'octa core' in processor.lower() else 0]\n"

In [291]:
# User Input Function
def get_user_input():
    print("\nPlease enter your laptop preferences:")
    ram = input("Enter RAM size (e.g., 8GB, 16GB): ").strip().lower()
    processor = input("Enter Processor type (e.g., i5, Ryzen 5): ").strip().lower()
    os = input("Enter Operating System (e.g., Windows, Linux): ").strip().lower()
    storage = input("Enter Storage size (e.g., 512GB SSD, 1TB HDD): ").strip().lower()
    laptop_brand = input("Enter Laptop Brand (e.g., Dell, HP): ").strip().lower()
    return {
        'ram': ram,
        'processor': processor,
        'os': os,
        'storage': storage,
        'laptop_brand': laptop_brand
    }

In [292]:
from sklearn.exceptions import NotFittedError


def recommend_laptop():
    try:
        user_input = get_user_input()
        user_df = pd.DataFrame([user_input])
        user_encoded = pd.get_dummies(user_df, drop_first=True)
        # Align columns with training data
        user_encoded = user_encoded.reindex(columns=X_encoded.columns, fill_value=0)
        prediction = rf_model.predict(user_encoded)
        print(f"\nRecommended laptop use case: {prediction[0]}")
    except NotFittedError:
        print("The model is not trained yet. Train the model before making predictions.")
    except ValueError as e:
        print(f"An error occurred during prediction: {e}")

In [293]:
recommend_laptop()


Please enter your laptop preferences:


NameError: name 'X_encoded' is not defined

In [ ]:
# Example: User selects a laptop with 8GB RAM and Intel i5 processor
ram = 64  # Example RAM size
processor = 'i5'  # Example processor
os = 'windows'

# Get the predicted course
predicted_usecase = predict_usecase(ram, processor)

# Output the predicted course
print(f"The recommended usecase for this laptop is: {predicted_usecase}")

TypeError: predict_usecase() missing 1 required positional argument: 'os'

In [348]:
rf_model

RandomForestClassifier(random_state=100)